In [ ]:
%matplotlib widget

import numpy as np
import yaml
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

In [ ]:
uploader = widgets.FileUpload(accept='.yml', multiple=False)
display(uploader)

In [ ]:
calib = yaml.safe_load(uploader.value[0].content.tobytes())
camera = calib[0]['camera']
focus = camera['focus']
sx = camera['sx']
sy = camera['sy']
cx = camera['cx']
cy = camera['cy']
k1 = camera['k1']
k2 = camera['k2']
k3 = camera['k3']
p1 = camera['p1']
p2 = camera['p2']
image_width = camera['image_width']
image_height = camera['image_height']

observ_points = np.array(calib[0]['calib_obj']['observ_points'])
object_points = np.array(calib[0]['calib_obj']['object_points'])
reproject_points = np.array(calib[0]['calib_obj']['reproject_points'])
rvecs = np.array(calib[0]['calib_obj']['rvecs'])
tvecs = np.array(calib[0]['calib_obj']['tvecs'])

npos = len(object_points)

In [ ]:
npos = 10
int_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=npos,
    step=1,
    description='Pose:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(int_slider)

In [ ]:
def draw(pos):
    plt.plot(observ_points[pos, ..., 0], observ_points[pos, ..., 1])
    plt.show()

In [ ]:
interact(draw, pos=(0, npos - 1))